In [274]:
#imports

import pandas as pd
import numpy as np
import math
import geopandas as gpd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures


pd.options.display.max_columns =200
pd.options.display.max_rows =500
pd.options.display.max_colwidth =500


#from seaborn import palplot

In [275]:
%matplotlib inline

from shapely.geometry import Point, Polygon
import matplotlib.pyplot as plt
import geopandas as gpd

In [11]:
train = pd.read_csv('train.csv', encoding='windows-1255')

In [12]:
train.head()

,trip_index,bus_id,time_recorded,lat,lon,route_id,line_index
0,3,7508169,18:54:00,32.054939,34.779751,20920,169
1,3,7508169,18:58:01,32.055996,34.779331,20920,173
2,3,7508169,18:59:17,32.056160,34.778408,20920,174
3,3,7508169,19:00:18,32.056400,34.780128,20920,175
4,3,7508169,19:03:13,32.061306,34.784782,20920,178


In [15]:
train.route_id.unique()

array([20920, 20916,  5189,  5214, 20912], dtype=int64)

In [121]:
route_20916 = train[train.route_id == 20916]
route_20916 = route_20916[route_20916.lon > 31]
route_20916 = route_20916[route_20916.lat > 31]
route_20916.head()

,trip_index,bus_id,time_recorded,lat,lon,route_id,line_index
67,4,7793169,06:18:52,32.055027,34.779896,20916,248
68,4,7793169,06:20:01,32.055798,34.779331,20916,249
69,4,7793169,06:22:05,32.056053,34.778351,20916,251
70,4,7793169,06:23:06,32.056026,34.781330,20916,252
71,4,7793169,06:25:36,32.060577,34.784698,20916,255


In [122]:
route_20916_X = route_20916.lat.values.reshape(-1, 1)
route_20916_y = route_20916.lon.values
len(route_20916_y)

17837

In [123]:
poly_features = PolynomialFeatures(degree=9)
X_poly_20916 = poly_features.fit_transform(route_20916_X)

In [124]:
def route_stats_2_stops(route_stats):
    
    route_stats = route_stats.copy()
    
    route_stats_stops = pd.DataFrame(columns = ['route_id','stop_index','stop_id','coordinates'])
    
    route_stats['all_stop_latlon'] = route_stats.apply(lambda r: route_stats_split_coordinates(r), axis=1)
    route_stats['all_stop_id'] = route_stats.apply(lambda r: route_stats_split_stops(r, 'all_stop_id'), axis=1)
    
    for route_id in route_stats['route_id']:
        route_id_stops = pd.DataFrame()
        route_id_stops['stop_id'] = pd.Series(route_stats[route_stats['route_id']==route_id]['all_stop_id'].tolist()[0])
        route_id_stops['coordinates'] = pd.Series(route_stats[route_stats['route_id']==route_id]['all_stop_latlon'].tolist()[0])
        route_id_stops['stop_index'] = route_id_stops.index+1
        route_id_stops['route_id'] = route_id
        
        route_stats_stops = route_stats_stops.append(route_id_stops)
    
    route_stats_stops = route_stats_stops[['route_id','stop_index','stop_id','coordinates']]\
            .sort_values(by = ['route_id','stop_index'])
    
    return route_stats_stops

#### plot with geopandas

In [262]:
def plot_route(route_stats_stops):
    route_stats_stops['coordinates'] = route_stats_stops['coordinates'].apply(Point)
    gdf = gpd.GeoDataFrame(route_stats_stops, geometry='coordinates')
    
    route_ids = dict(tuple(gdf.groupby('route_id')))
    
    fig = plt.figure(num=None, figsize=(10, 8))
    
    first_route_id = list(route_ids.keys())[0]
    
    ax = route_ids[first_route_id].plot()
    
    for route_id in route_ids.keys():
        if route_id == first_route_id:
            pass
        else:
            route_ids[route_id].plot(ax=ax)

In [264]:
plot_route(df_plot)
#plt.figure(num=None, figsize=(10, 8))

KeyError: 'figure.constrained_layout.use'

In [221]:
lr_model = LinearRegression()

In [222]:
lr_model.fit(X_poly_20916, route_20916_y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [226]:
def plot_poly(model, poly_features):
    x_plot = np.linspace(32.05491257, 32.1930275, 100) #min(x), max(x), 100)
    X_plot_poly = poly_features.fit_transform(x_plot.reshape(-1, 1))
    y_plot = model.predict(X_plot_poly)
    df_plot = pd.DataFrame({'lat': x_plot, 'lon': y_plot})
    #plot_route(df_plot)
    df_plot['coordinates'] = df_plot['coordinates'].apply(Point)
    gdf = gpd.GeoDataFrame(df_plot, geometry='coordinates')
    
    route_ids = dict(tuple(gdf.groupby('route_id')))
    
    fig = plt.figure(num=None, figsize=(10, 8))
    
    first_route_id = list(route_ids.keys())[0]
    
    ax = route_ids[first_route_id].plot()
    
    for route_id in route_ids.keys():
        if route_id == first_route_id:
            pass
        else:
            route_ids[route_id].plot(ax=ax)



In [227]:
def plot_data(x, y):
    plt.scatter(x, y)
    plt.xlabel('Latitude')
    plt.ylabel('Longtitude')

In [233]:
plot_poly(lr_model, poly_features)

TypeError: must be real number, not str

In [254]:
df_plot = pd.DataFrame({'lat': route_20916.lat.values, \
                        'lon': route_20916.lon.values, 'route_id': route_20916.route_id.values})
df_plot['coordinates'] = list(zip(df_plot.lat, df_plot.lon))
#df_plot['coordinates'] = df_plot['coordinates'].apply(Point)
#gdf = gpd.GeoDataFrame(df_plot, geometry='coordinates')

In [276]:
a = [1, 2, 3]
b = [4,5,6]
plt.plot(a, b)

KeyError: 'figure.constrained_layout.use'